In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

# 1. CẤU HÌNH FILE
input_files = {
    "train": "/content/drive/MyDrive/NLP/raw_data/train.jsonl",
    "dev": "/content/drive/MyDrive/NLP/raw_data/dev.jsonl",
    "test": "/content/drive/MyDrive/NLP/raw_data/test.jsonl"
}

output_files = {
    "train": "/content/drive/MyDrive/NLP/data_llm/train_llm.jsonl",
    "dev": "/content/drive/MyDrive/NLP/data_llm/dev_llm.jsonl",
    "test": "/content/drive/MyDrive/NLP/data_llm/test_llm.jsonl"
}

# 2. THIẾT KẾ PROMPT CHI TIẾT (Instruction Engineering)
INSTRUCTION = """Bạn là một chuyên gia AI về phân tích cảm xúc khách hàng (ABSA). Nhiệm vụ của bạn là đọc bình luận và trích xuất các khía cạnh (Aspect) cùng cảm xúc (Sentiment) và nội dung tương ứng (Span).

DANH SÁCH NHÃN:
- Aspect: [SCREEN, CAMERA, FEATURES, BATTERY, PERFORMANCE, STORAGE, DESIGN, PRICE, GENERAL, SER&ACC]
- Sentiment: [POSITIVE, NEGATIVE, NEUTRAL]

HƯỚNG DẪN GÁN NHÃN QUAN TRỌNG:
1. SER&ACC (Dịch vụ & Phụ kiện): Bao gồm CSKH, bảo hành, giao hàng, đóng gói, và các phụ kiện đi kèm (sạc, cáp, tai nghe...).
   - Ví dụ: "Giao hàng nhanh" -> SER&ACC#POSITIVE. "Sạc lỏng lẻo" -> SER&ACC#NEGATIVE.
2. FEATURES (Tính năng): Các tính năng cụ thể như Vân tay, FaceID, Wifi, 4G, Bluetooth, Sim, Chống nước...
   - Ví dụ: "Vân tay nhạy" -> FEATURES#POSITIVE. "Bắt wifi kém" -> FEATURES#NEGATIVE.
3. QUY TẮC NHÃN 'GENERAL' (Tổng quan):
   - GENERAL#POSITIVE: Nếu (Tổng số nhãn Positive khác) - (Tổng số nhãn Negative khác) >= 2.
   - GENERAL#NEGATIVE: Nếu (Tổng số nhãn Negative khác) - (Tổng số nhãn Positive khác) >= 2.
   - GENERAL#NEUTRAL: Các trường hợp còn lại (chênh lệch <= 1) hoặc khi khách hàng nhận xét chung chung "Máy ổn", "Tạm được".

ĐỊNH DẠNG OUTPUT:
Trả về JSON list chứa các object: {"aspect": "...", "sentiment": "...", "span": "..."}. Nếu không có thông tin, trả về []."""

# 3. HÀM CHUYỂN ĐỔI DỮ LIỆU
def convert_index_format_to_llm_v2(raw_path, output_path):
    if not os.path.exists(raw_path):
        print(f"Không thấy file.")
        return

    with open(raw_path, 'r', encoding='utf-8') as f_in, \
         open(output_path, 'w', encoding='utf-8') as f_out:

        count = 0
        for line in f_in:
            try:
                # Đọc dữ liệu thô
                data = json.loads(line.strip())
                text = data.get('text', '')
                raw_labels = data.get('labels', [])

                output_list = []

                # Xử lý từng nhãn trong list
                for label in raw_labels:
                    if len(label) == 3:
                        start_idx, end_idx, tag_str = label

                        # 1. Cắt Span từ Text
                        if start_idx < len(text) and end_idx <= len(text):
                            span_text = text[start_idx:end_idx]
                        else:
                            span_text = "ERROR_SPAN" # Đánh dấu nếu lỗi index

                        # 2. Tách Aspect và Sentiment
                        # VD: "BATTERY#POSITIVE" -> Aspect: BATTERY, Sent: POSITIVE
                        if "#" in tag_str:
                            aspect, sentiment = tag_str.split("#", 1)
                        else:
                            aspect = tag_str
                            sentiment = "UNKNOWN"

                        output_list.append({
                            "aspect": aspect,
                            "sentiment": sentiment,
                            "span": span_text
                        })

                # Chuyển list kết quả thành chuỗi JSON
                output_str = json.dumps(output_list, ensure_ascii=False)

                # Tạo bản ghi Alpaca Format
                llm_record = {
                    "instruction": INSTRUCTION, # Prompt mới chứa quy tắc General/Rare
                    "input": text,
                    "output": output_str
                }

                # Ghi xuống file
                f_out.write(json.dumps(llm_record, ensure_ascii=False) + "\n")
                count += 1

            except Exception as e:
                print(f"Lỗi dòng {line[:30]}... -> {e}")

    print(f"File lưu tại: {output_path} ({count} mẫu)")

# 4. CHẠY CODE
for split in input_files:
    convert_index_format_to_llm_v2(input_files[split], output_files[split])